In [1]:
%%time
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
import tools.Sample_Tools as smpl
import tools.Factor_Cacher as cacher
import tools.Pretreat_Tools as pretreat
from base.JuUnits import excute_for_multidates

import pandas as pd
import numpy as np



Wall time: 2.16 s


In [4]:
%%time
factors = cacher.load_caches_adv(['alpha11','alpha24','alpha38','alpha41'])
codes = factors.index.get_level_values(1).unique()
dates = factors.index.get_level_values(0).unique()
data = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
data = smpl.add_marketvalue_industry(data)
ret_forward = smpl.get_forward_return(data,'close')
ret_forward.name = 'ret_forward'
factors = factors.assgin(country=1, industry = data['industry'])
factors = pd.get_dummies(factors,'industry')


Wall time: 29.3 s


In [73]:
# factors.info(memory_usage='deep')
# factors.memory_usage(deep=True)
# import gc
# gc.collect()

167423